### TensorFlow-Slim image classification model library

https://github.com/tensorflow/models/tree/master/slim

TF-slim is a new lightweight high-level API of TensorFlow (tensorflow.contrib.slim) for defining, training and evaluating complex models. This directory contains code for training and evaluating several widely used Convolutional Neural Network (CNN) image classification models using TF-slim.

- train models from scratch or fine-tune them from pre-trained network weights
- downloading standard image datasets (Flowers, MNIST, cifar10, ImageNet), converting them to TensorFlow's native TFRecord format


In [ ]:
%matplotlib inline

import os
import sys
import math
import numpy as np
import tensorflow as tf

sys.path.insert(0,'../models/slim')
from datasets import download_and_convert_cifar10
from datasets import download_and_convert_flowers
from datasets import download_and_convert_mnist

_DATASET_NAME='cifar10'
dataset_dir = '/tmp/datasets/' + _DATASET_NAME
download_and_convert_cifar10.run(dataset_dir)

### Downloading 'cifar10' and Converting to TFRecord format

download 'cifar-10-python' dataset from 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz' and convert to TFRecord format. 